In [ ]:
# Importar as bibliotecas.
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
import os
from os import listdir
from os.path import isfile, join
import statsmodels.api as sm
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image

In [ ]:
# Importar o ficheiro que contém a informação dos países presentes na base de dados, e que contém 2 colunas que serão utilizadas no decorrer do trabalho (Region e IncomeGroup).
metadata = pd.read_csv('Metadata_Country_API_SE.XPD.TOTL.GD.ZS_DS2_en_csv_v2_511370.csv')
metadata.head()

In [ ]:
# Verificar a  quantidade de linhas e colunas.
metadata.shape

In [ ]:
# Dividir o ficheiro metadata, de modo a reter as colunas pretendidas.
dataframe = metadata.iloc[:,0:3]
dataframe.head()

In [ ]:
# Criar uma lista com todos os ficheiros CSV a serem considerados na análise. Os ficheiros devem estar guardados na mesma pasta que o Jupyter Notebook.

inputfiles = [f for f in listdir(os.getcwd()) if (f.endswith(".csv") and f.startswith("A")) if isfile(join(os.getcwd(), f))]

# Iniciar as variáveis em que os DataFrames serão armazenados.

df0, df1=[0]*len(inputfiles), [0]*len(inputfiles)

# Deixar o utilizador decidir o ano para o qual a análise será realizada.

user_yr=int(input("Qual é o ano que deseja analisar? "))

#Loop
for f in inputfiles:
    # Ler o conteúdo dos ficheiros na lista df0.
    df0[inputfiles.index(f)]=pd.read_csv(f, skiprows=4)
    # Os ficheiros contêm dados que duram quase 60 anos. A variável seguinte determina a coluna, do ficheiro, que possui os dados correspondentes ao ano escolhido.
    i_yr=user_yr-1960-(len(df0[inputfiles.index(f)].columns)-4)
    # Criar um novo DataFrame com as colunas relevantes para a análise - mantivemos o código do país (para poder encontrar uma correspondência entre todos os dicheiros) e o nome de cada indicador (para nomear a coluna correta e automaticamente).
    df1[inputfiles.index(f)] = df0[inputfiles.index(f)].iloc[:,[1,2,i_yr]]
    # Renomear a coluna que contêm a data, em df1, pelo indicador.
    df1[inputfiles.index(f)] = df1[inputfiles.index(f)].rename(columns={df1[inputfiles.index(f)].columns[2]: df1[inputfiles.index(f)].iloc[0,1]})
    # Eliminar a coluna cujo nome é "Indicator".
    df1[inputfiles.index(f)] = df1[inputfiles.index(f)].loc[:, ~df1[inputfiles.index(f)].columns.str.startswith('Indicator')]

In [ ]:
# Juntar todos os dados, para o ano escolhido, num único DataFrame. Esta ação é feita através da variável "Country Code" para que exista uma correnpondência entre todos os ficheiros.
df=df1[0]
for i in range(1,len(df1)):
    df=pd.merge(df1[i], df, how='left',left_on='Country Code', right_on='Country Code')
df.head()

In [ ]:
# Verificar a quantidade de linhas e colunas para o ano escolhido.
df.shape

In [ ]:
# Visualizar um país, neste caso, França. 
df.loc[df['Country Code'] == "FRA"]

In [ ]:
# Montar a estrutura da tabela de objetos (left): Country Name, Country Code, Region, IncomeGroup.
dataframe = pd.merge(dataframe,df, how='left',left_on='Country Code', right_on='Country Code')
dataframe.tail()

In [ ]:
# Eliminar as linhas que contêm erros, ou dados omissos, nalguma das variáveis.
dataframe=dataframe.dropna()
dataframe.head()

In [ ]:
# Verificar a quantidade de linhas e colunas, após a eliminação das linhas.
dataframe.shape

In [ ]:
# Visualizar os ultimos registos.
dataframe.tail(4)

In [ ]:
# Renumerar o indice do dataframe.
dataframe = dataframe.reset_index(drop=True)
dataframe.head(3)

In [ ]:
# Verificar o tipo das variáveis presente no dataframe.
dataframe.dtypes

In [ ]:
# Verificar a correlação entre as variáveis presentes no dataframe.
fig = plt.figure(figsize=[8, 8])
corr_mtx = dataframe.corr()
sns.heatmap(corr_mtx, xticklabels=corr_mtx.columns, yticklabels=corr_mtx.columns, annot=True, cmap='Blues')
plt.title('Correlation analysis')
plt.show()

In [ ]:
# Eliminar variáveis indejadas, caso existam, devido à elevada correlação. Responder "Não" caso não deseje executar uma eliminação e "Sim" no caso de desejar.
a=input("Gostaria de eliminar alguma variável do dataframe? ")
if a=="Não":
    print("OK!")
elif a=="Sim":
    b=input("Qual é o nome da coluna que deseja eliminar? ")
    dataframe1=dataframe.drop(b,axis=1)
    dataframe1.head()
else:
    print("Tente outra vez!")

In [ ]:
# Definir a variável dependente.
Y = dataframe.loc[:,'GDP per capita (current US$)']

In [ ]:
# Definir as variáveis explicativas.
X = dataframe.drop(['Country Code','Region', 'IncomeGroup', 'GDP per capita (current US$)'], axis=1)

In [ ]:
# Regresão em que a variável dependente é "GDP per capita (current US$)" e as restantes variáveis são as explicativas.
X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()
model.summary()

In [ ]:
# Criar scatterplot para a relação entre o GDP per capita e a variável com um maior impacto.
i=input("Qual é a variável com maior coeficente? ")
ydata = Y
xdata = X.loc[:,i]
colors = (0,0,0)
area = np.pi*3
fig = plt.figure(figsize=(20,10))
plt.scatter(
    xdata, 
    ydata, 
    s = area, 
    c = colors, 
    alpha = 1)

for label, x, y in zip(dataframe.iloc[:,0], xdata, ydata):
    plt.annotate(
        label,
        xy=(x, y), xytext=(2, 0),
        textcoords = 'offset points', ha = 'left', va = 'bottom',
        #bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
        #arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0')
    )
    

plt.title('Scatter plot')
plt.xlabel(i)
plt.ylabel('GDP per capita (current US$)')
plt.show()

In [ ]:

# Criar uma scatter matrix para analisar a relação, dois a dois, das variáveis.
stats = dataframe1.columns[3:]
corr = dataframe1[stats].corr() # corr é utilizado para encontrar a correlação.
color_function = {0: 'blue', 1: 'red'}
colors = dataframe1['GDP per capita (current US$)'].map(lambda x: color_function.get(x))
scatter_matrix(dataframe1[stats], alpha = 1, figsize = (25, 25))
plt.show()

In [ ]:
# Dividir o dataframe para ficar apenas com os valores e os índices.
dataframe2 = dataframe.iloc[:,3:]
dataframe2.head()

In [ ]:
# Aplicar o Metodo Elbow para encontrar o número de clusters a considerar na análise.
wcss = []
for i in range(1, 10):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=3000, n_init=10, random_state=0)
    kmeans.fit(dataframe2)
    wcss.append(kmeans.inertia_)
plt.plot(range(1, 10), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
# Definir o número de clusters que serão considerdados na análise.
clus_num = int(input("De acordo com o método Elbow, quantos Clusters quer considerar na sua análise? "))
kmeans = KMeans(n_clusters = clus_num).fit(dataframe2)
centroids = kmeans.cluster_centers_
print(centroids)

In [ ]:
pred_y = kmeans.predict(dataframe2)
print(pred_y)

In [ ]:
pred_y.shape

In [ ]:
# Criar o dataframe com os grupos (Clusters).
groups = pd.DataFrame(pred_y, columns = ['Group'])
groups.head()

In [ ]:
# Adicionar a coluna que contém o grupo do país ao dataframe inicial.
dataframe = dataframe.merge(groups,left_index=True, right_index=True)
dataframe.head(2)

In [ ]:
# Exibir o gráfico com a contabilização de quantos países pertencem a cada grupo, por região.
sns.countplot(y='Region', hue = 'Group', data = dataframe)

In [ ]:
# Criar um novo data frame para a Decision Tree.
ndataframe = ['Foreign direct investment, net inflows (% of GDP)','Gross savings (% of GDP)','Expense (% of GDP)','GDP per capita (current US$)','Exports of goods and services (% of GDP)','Inflation, consumer prices (annual %)']
# Separar por variaveis.
nx = dataframe.loc[:,ndataframe].values
# Separar o alvo de estudo.
ny = dataframe.loc[:,['IncomeGroup']].values

In [ ]:
# Dividir o dataframe entre treino e teste.
nx_train, nx_test, ny_train, ny_test = train_test_split(nx, ny, test_size=0.3, random_state=1)

In [ ]:
# Criar o objeto da Decision Tree Classifer.
clf = DecisionTreeClassifier()
# Treinar Decision Tree Classifer.
clf = clf.fit(nx_train,ny_train)
# Prever a resposta para o dataframe de teste.
y_pred1 = clf.predict(nx_test)

In [ ]:
# Encontrar a taxa de precisão do modelo.
print("A taxa de precisão do modelo é de:",metrics.accuracy_score(ny_test, y_pred1))

In [ ]:
# Criar o arquivo com a Decision Tree.
import pydotplus
dot_data = StringIO()
export_graphviz(clf, out_file = dot_data, filled = True, rounded = True,
                special_characters = True, feature_names = ndataframe)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('IncomeGroupDT.png')
Image(graph.create_png())